In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import time

In [ ]:
#Montagem do Drive Google 
from google.colab import drive 
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


Documento explicativo acerca do preenchimento do SIH

http://repositorio.ipea.gov.br/bitstream/11058/9409/1/Uma_analise_da_base_de_dados_do_sistema_de_informacao_hospitalar.pdf

In [ ]:
#Definindo colunas úteis e dicionário com dtypes do dataset SIH

dtype = np.dtype
colunashosp = ["N_AIH","MUNIC_RES", "CEP", "SEXO", "IDADE","NASC",
                "RACA_COR",  "CNES", "PROC_REA", "VAL_TOT", "US_TOT", "DT_INTER", "DT_SAIDA",
               'QT_DIARIAS',"DIAS_PERM","DIAG_PRINC",  "DIAGSEC1", 
               "DIAGSEC2", "DIAGSEC3"]

hosp_dict_types={'N_AIH': dtype('O'),'MUNIC_RES': dtype('O'),"CEP":dtype('O'), 'SEXO': dtype('O'), 
                   'RACA_COR': dtype('O'),"CNES": dtype('O'), 'PROC_REA': dtype("O"), 'VAL_TOT': dtype('float64'),'US_TOT': dtype('float64'),
                 'DT_INTER': dtype('datetime64'), 'DT_SAIDA': dtype('datetime64'),'QT_DIARIAS': dtype('int32'),'DIAG_PRINC': dtype('O'), 'DIAS_PERM': dtype('int32'),
                 'IDADE': dtype('int32'), 'NASC': dtype('datetime64'), 'DIAGSEC1': dtype('O'), 
                 'DIAGSEC2': dtype('O'), 'DIAGSEC3': dtype('O')}

#Valores úteis
DicioRacacor = {"01":"BRANCA", 
                "02": "PRETA", 
                "03": "PARDA", 
                "04": "AMARELA", 
                "05": "INDÍGENA", 
                "99": "S/ INFO",
                "00": "00"}
DicioSexo = {"1": "Masculino", "3": "Feminino"}                       

In [ ]:
#Célula para criar o csv geral hospitalar. Executar apenas se necessário refazer


path_sih = '/content/gdrive/MyDrive/DERMATITE_ATOPICA/restante_arquivos_SIH'
filelist = [i for i in os.listdir(path_sih)] 

tempdf = pd.DataFrame(columns=colunashosp)
df = pd.DataFrame(columns=colunashosp)
#Criando filtros para selecionar ocorrências de "L"  nas colunas com CID presente

cache = {}
cache["Registros"] = np.array([0])


for i in filelist:
  if int(i[5:10])%(30) == 0:
    print(f"Arquivo lido: {i}")   
  tempdf = pd.read_parquet(path_sih+"/"+i, engine='pyarrow', columns= colunashosp)
  maskl_hosp = (tempdf["DIAG_PRINC"].str.contains("L") | tempdf["DIAGSEC1"].str.contains("L")
             | tempdf["DIAGSEC2"].str.contains("L") | tempdf["DIAGSEC3"].str.contains("L"))
  cache["Registros"] += np.array(len(tempdf))
  tempdf = tempdf[maskl_hosp]
  
  
  df = df.append(tempdf.astype(hosp_dict_types))
  
#df.set_index("MUNIC_RES").to_csv('/content/gdrive/MyDrive/DERMATITE_ATOPICA/L_Hosp_dataset_FINAL_3.csv')  


Arquivo lido: part-00000-b4b3bdd6-7597-4295-bfbe-ef87752d34ab-c000.snappy.parquet
Arquivo lido: part-00030-b4b3bdd6-7597-4295-bfbe-ef87752d34ab-c000.snappy.parquet
Arquivo lido: part-00060-b4b3bdd6-7597-4295-bfbe-ef87752d34ab-c000.snappy.parquet
Arquivo lido: part-00090-b4b3bdd6-7597-4295-bfbe-ef87752d34ab-c000.snappy.parquet
Arquivo lido: part-00120-b4b3bdd6-7597-4295-bfbe-ef87752d34ab-c000.snappy.parquet
Arquivo lido: part-00150-b4b3bdd6-7597-4295-bfbe-ef87752d34ab-c000.snappy.parquet
Arquivo lido: part-00180-b4b3bdd6-7597-4295-bfbe-ef87752d34ab-c000.snappy.parquet
Arquivo lido: part-00210-b4b3bdd6-7597-4295-bfbe-ef87752d34ab-c000.snappy.parquet
Arquivo lido: part-00240-b4b3bdd6-7597-4295-bfbe-ef87752d34ab-c000.snappy.parquet
Arquivo lido: part-00270-b4b3bdd6-7597-4295-bfbe-ef87752d34ab-c000.snappy.parquet
Arquivo lido: part-00300-b4b3bdd6-7597-4295-bfbe-ef87752d34ab-c000.snappy.parquet


In [ ]:
#Número de Registros hospitalares lidos
cache

{'Registros': array([56876187])}

Foram lidos 341 arquivos parquet, resultando em 56.876.187 linhas

---



In [ ]:
#Veririficação do Dataset formado
df = pd.read_csv('/content/gdrive/MyDrive/DERMATITE_ATOPICA/L_Hosp_dataset_FINAL_3.csv')
df.head()

,MUNIC_RES,N_AIH,CEP,SEXO,IDADE,NASC,RACA_COR,CNES,PROC_REA,VAL_TOT,US_TOT,DT_INTER,DT_SAIDA,QT_DIARIAS,DIAS_PERM,DIAG_PRINC,DIAGSEC1,DIAGSEC2,DIAGSEC3
0,355030,3517114674887,5857480,1,49,1968-04-10,99,2091313,301060088,44.22,13.40,2017-05-27,2017-05-28,1,1,L030,NaN,NaN,NaN
1,352310,3517113941913,8575230,1,33,1983-06-26,1,2091542,401020100,158.11,47.91,2017-05-12,2017-05-12,0,0,L989,NaN,NaN,NaN
2,355030,3517113941968,2226000,3,40,1976-08-15,2,2091542,401020100,158.11,47.91,2017-05-12,2017-05-12,0,0,L989,NaN,NaN,NaN
3,355030,3517114189006,3975080,1,50,1966-10-02,3,2091585,401020070,167.72,50.82,2017-05-03,2017-05-03,1,0,L720,NaN,NaN,NaN
4,355030,3517108978064,3678030,3,45,1972-03-17,1,2091542,401020100,158.11,47.91,2017-05-16,2017-05-16,0,0,L989,NaN,NaN,NaN


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1288483 entries, 0 to 1288482
Data columns (total 19 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   MUNIC_RES   1288483 non-null  int64  
 1   N_AIH       1288483 non-null  int64  
 2   CEP         1288483 non-null  int64  
 3   SEXO        1288483 non-null  int64  
 4   IDADE       1288483 non-null  int64  
 5   NASC        1288483 non-null  object 
 6   RACA_COR    1288483 non-null  int64  
 7   CNES        1288483 non-null  int64  
 8   PROC_REA    1288483 non-null  int64  
 9   VAL_TOT     1288483 non-null  float64
 10  US_TOT      1288483 non-null  float64
 11  DT_INTER    1288483 non-null  object 
 12  DT_SAIDA    1288483 non-null  object 
 13  QT_DIARIAS  1288483 non-null  int64  
 14  DIAS_PERM   1288483 non-null  int64  
 15  DIAG_PRINC  1288483 non-null  object 
 16  DIAGSEC1    177270 non-null   object 
 17  DIAGSEC2    38500 non-null    object 
 18  DIAGSEC3    15461 non-

In [ ]:
#Prévia das ocorrências de CID L20
maskl20_hosp = (df["DIAG_PRINC"].str.contains("L20") | df["DIAGSEC1"].str.contains("L20")
               | df["DIAGSEC2"].str.contains("L20")  | df["DIAGSEC3"].str.contains("L20"))
dfl20 = df[maskl20_hosp]
dfl20

,MUNIC_RES,N_AIH,CEP,SEXO,IDADE,NASC,RACA_COR,CNES,PROC_REA,VAL_TOT,US_TOT,DT_INTER,DT_SAIDA,QT_DIARIAS,DIAS_PERM,DIAG_PRINC,DIAGSEC1,DIAGSEC2,DIAGSEC3
146,350950,3517101072090,13069096,1,8,2016-08-26,1,2079798,303140135,781.30,236.75,2017-05-08,2017-05-14,6,6,J960,J218,L209,NaN
252,355030,3517108914594,5405150,1,5,2011-12-10,3,2078015,303140046,761.83,230.85,2017-03-27,2017-03-29,2,2,J46,J304,L209,NaN
263,355030,3517108318207,2992060,1,12,2004-11-12,1,2078015,301060010,55.27,16.74,2017-05-11,2017-05-12,1,1,L209,NaN,NaN,NaN
780,351070,3517113622869,15570000,1,5,2011-11-05,3,2082071,303080051,240.91,73.00,2017-05-16,2017-05-18,2,2,L208,NaN,NaN,NaN
941,355030,3517108920391,4932310,1,4,2012-11-01,1,2078015,303080051,264.91,80.27,2017-03-31,2017-04-05,5,5,L208,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1287462,160030,1617100399850,68909171,1,25,1992-05-18,99,2020653,303080051,655.66,198.68,2017-10-29,2017-11-17,19,19,L208,NaN,NaN,NaN
1287549,120060,1219100060222,69970000,1,10,2008-12-09,3,2000121,303080051,240.91,61.93,2019-01-13,2019-01-16,3,3,L208,NaN,NaN,NaN
1287597,130356,1219100064248,69117000,3,10,2008-08-15,3,5336171,303080051,256.91,66.04,2019-02-24,2019-02-28,4,4,L209,NaN,NaN,NaN
1288091,160030,1616100291300,68903188,1,36,1980-02-12,99,2020653,303080051,303.62,97.31,2016-12-16,2016-12-19,3,3,L208,NaN,NaN,NaN
